# 네이버 뉴스 기사 카테고리 분류하기

-----

In [1]:
from sklearn.datasets import load_files
import pandas as pd
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import pickle
import re
import konlpy
import nltk
import numpy as np
import os
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\momo1\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

### 뉴스기사 데이터 불러오기.
- `from sklearn.datasets import load_files`

In [2]:
naver_news = load_files('../data/newsData', shuffle=True)
naver_news.keys()

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])

In [3]:
# 각 뉴스의 본문은 바이트 코드로 저장이 되어있습니다.
naver_news.data[0]

b'\xed\x83\x9c\xec\x96\x91\xea\xb4\x91\xec\x9c\xbc\xeb\xa1\x9c \xec\xb6\xa9\xec\xa0\x84\xeb\x90\x98\xeb\x8a\x94 \xe2\x80\x98\xec\x95\x84\xec\x9d\xb4\xed\x8f\xb0X \xed\x85\x8c\xec\x8a\xac\xeb\x9d\xbc\xe2\x80\x99\t(\xec\xa7\x80\xeb\x94\x94\xeb\x84\xb7\xec\xbd\x94\xeb\xa6\xac\xec\x95\x84=\xec\x9d\xb4\xec\xa0\x95\xed\x98\x84 \xea\xb8\xb0\xec\x9e\x90)\xeb\x8f\x84\xeb\x84\x90\xeb\x93\x9c \xed\x8a\xb8\xeb\x9f\xbc\xed\x94\x84 \xeb\xaf\xb8\xea\xb5\xad \xeb\x8c\x80\xed\x86\xb5\xeb\xa0\xb9\xea\xb3\xbc \xed\x91\xb8\xed\x8b\xb4 \xeb\x9f\xac\xec\x8b\x9c\xec\x95\x84 \xeb\x8c\x80\xed\x86\xb5\xeb\xa0\xb9\xec\x9d\x98 \xec\xb4\x88\xec\x83\x81\xed\x99\x94\xeb\xa5\xbc \xec\x83\x88\xea\xb8\xb4 \xed\x99\xa9\xea\xb8\x88 \xec\x95\x84\xec\x9d\xb4\xed\x8f\xb0\xec\x9d\x84 \xec\xb6\x9c\xec\x8b\x9c\xed\x95\xb4 \xed\x99\x94\xec\xa0\x9c\xeb\xa5\xbc \xeb\xaa\xa8\xec\x95\x98\xeb\x8d\x98 \xeb\x9f\xac\xec\x8b\x9c\xec\x95\x84 \xec\x97\x85\xec\xb2\xb4 \xec\xba\x90\xeb\xb9\x84\xec\x95\x84(Caviar)\xea\xb0\x80 \xec\x9d\xb4\xe

In [4]:
# 데이터의 target값은 총 8개로, 아래와 같습니다.
labels = ['정치', '경제', '사회', '생활/문화', '세계', '기술/IT', '연예', '스포츠']
naver_news.target_names

['0', '1', '2', '3', '4', '5', '6', '7']

In [5]:
# 뉴스 본문과 분류를 각각 처음부터 50건만 제한하여 X, y로 저장합니다.
X = naver_news.data
y = naver_news.target

print(X[0]) #b'\xed\x83\x9c\xec\x96\x91\xea\xb4\x91\xec\x9c\xbc\xeb\xa1\x9c \xec\xb6\xa9\xec\xa0\x84\xeb\x90\x98\xeb\x8a\x94 \xe2\x80\x98\xec\x95\x84\xec\x9d\xb4\xed\x8f\xb0X ...'
print(y[0]) #5

b'\xed\x83\x9c\xec\x96\x91\xea\xb4\x91\xec\x9c\xbc\xeb\xa1\x9c \xec\xb6\xa9\xec\xa0\x84\xeb\x90\x98\xeb\x8a\x94 \xe2\x80\x98\xec\x95\x84\xec\x9d\xb4\xed\x8f\xb0X \xed\x85\x8c\xec\x8a\xac\xeb\x9d\xbc\xe2\x80\x99\t(\xec\xa7\x80\xeb\x94\x94\xeb\x84\xb7\xec\xbd\x94\xeb\xa6\xac\xec\x95\x84=\xec\x9d\xb4\xec\xa0\x95\xed\x98\x84 \xea\xb8\xb0\xec\x9e\x90)\xeb\x8f\x84\xeb\x84\x90\xeb\x93\x9c \xed\x8a\xb8\xeb\x9f\xbc\xed\x94\x84 \xeb\xaf\xb8\xea\xb5\xad \xeb\x8c\x80\xed\x86\xb5\xeb\xa0\xb9\xea\xb3\xbc \xed\x91\xb8\xed\x8b\xb4 \xeb\x9f\xac\xec\x8b\x9c\xec\x95\x84 \xeb\x8c\x80\xed\x86\xb5\xeb\xa0\xb9\xec\x9d\x98 \xec\xb4\x88\xec\x83\x81\xed\x99\x94\xeb\xa5\xbc \xec\x83\x88\xea\xb8\xb4 \xed\x99\xa9\xea\xb8\x88 \xec\x95\x84\xec\x9d\xb4\xed\x8f\xb0\xec\x9d\x84 \xec\xb6\x9c\xec\x8b\x9c\xed\x95\xb4 \xed\x99\x94\xec\xa0\x9c\xeb\xa5\xbc \xeb\xaa\xa8\xec\x95\x98\xeb\x8d\x98 \xeb\x9f\xac\xec\x8b\x9c\xec\x95\x84 \xec\x97\x85\xec\xb2\xb4 \xec\xba\x90\xeb\xb9\x84\xec\x95\x84(Caviar)\xea\xb0\x80 \xec\x9d\xb4\xe

---
## 데이터 분석
### 1. 데이터 전처리
- 바이트코드를 문자열로 변환합니다.

In [6]:
X = [bytecode.decode('utf-8') for bytecode in X] 
print(X[0]) #태양광으로 충전되는 ‘아이폰X 테슬라’ (지디넷코리아=이정현 기자)도널드 트럼프 미국 대통령과 푸틴 러시아 대통령의 초상화를 새긴 황금 아이폰을 출시해 ...

태양광으로 충전되는 ‘아이폰X 테슬라’	(지디넷코리아=이정현 기자)도널드 트럼프 미국 대통령과 푸틴 러시아 대통령의 초상화를 새긴 황금 아이폰을 출시해 화제를 모았던 러시아 업체 캐비아(Caviar)가 이번엔 태양광 충전이 가능한 아이폰X를 판매한다고 IT매체 더버지가 16일(현지시간) 보도했다.


이 업체는 주로 아이폰, 애플워치 등의 IT 제품들을 금으로 장식하고 개조해 고가에 판매하는 업체다.

이 아이폰의 이름은 ‘아이폰X 테슬라(Tesla)’. 이 제품의 뒷면에는 태양 전지 패널이 있어 스마트폰을 뒤집어 놓기만 하면 충전이 되어 보조 배터리 형태로 사용할 수 있다.


물론, 이 업체가 아이폰을 직접 만드는 것이 아니기 때문에 아이폰X에 탑재된 태양광 충전 기능은 아이폰에 직접 내장되어 있는 형태가 아닌 아이폰 바깥 쪽에 케이스 형태로 탑재된다. 또, 캐비아는 그 동안 제공했던 IT 제품들과 마찬가지로 고급스러움을 극대화 하기 위해 노력했다. 아이폰 뒷면 모서리와 후면 카메라 테두리 등 곳곳을 24k 금으로 장식했다.

이 제품은 999대 한정 생산된다. 64GB 모델 아이폰X 테슬라의 가격은 4,555달러(약 490만 원), 256GB모델의 경우 4,805달러(약 517만원)다.


#### 뉴스 본문에 정규식을 적용하여 한글 텍스트만 추출합니다.

In [7]:
# 한글 데이터만 추출합니다.
for i, news in enumerate(X):
    news = re.sub(r'[^ ㄱ-ㅣ가-힣]', '', news) #특수기호 제거, 정규 표현식
    X[i] = news

print(X[0])

태양광으로 충전되는 아이폰 테슬라지디넷코리아이정현 기자도널드 트럼프 미국 대통령과 푸틴 러시아 대통령의 초상화를 새긴 황금 아이폰을 출시해 화제를 모았던 러시아 업체 캐비아가 이번엔 태양광 충전이 가능한 아이폰를 판매한다고매체 더버지가 일현지시간 보도했다이 업체는 주로 아이폰 애플워치 등의제품들을 금으로 장식하고 개조해 고가에 판매하는 업체다이 아이폰의 이름은 아이폰 테슬라 이 제품의 뒷면에는 태양 전지 패널이 있어 스마트폰을 뒤집어 놓기만 하면 충전이 되어 보조 배터리 형태로 사용할 수 있다물론 이 업체가 아이폰을 직접 만드는 것이 아니기 때문에 아이폰에 탑재된 태양광 충전 기능은 아이폰에 직접 내장되어 있는 형태가 아닌 아이폰 바깥 쪽에 케이스 형태로 탑재된다 또 캐비아는 그 동안 제공했던제품들과 마찬가지로 고급스러움을 극대화 하기 위해 노력했다 아이폰 뒷면 모서리와 후면 카메라 테두리 등 곳곳을  금으로 장식했다이 제품은 대 한정 생산된다 모델 아이폰 테슬라의 가격은 달러약 만 원 모델의 경우 달러약 만원다


#### 형태소 분석 - 명사, 동사, 형용사만 추출합니다.
- konlpy를 사용해서, 형태소 단위로 문장을 분석합니다.

In [8]:
for i, news in enumerate(X):
    okt = konlpy.tag.Okt()
    clean_words = []
    for word in okt.pos(news, stem=True): # 어간 추출
        if word[1] in ['Noun', 'Verb', 'Adjective']: # 명사, 동사, 형용사 추출
            clean_words.append(word[0])
    news = ' '.join(clean_words)
    X[i] = news
    
print(X[0])

태양광 충전 되다 아이폰 테슬라 디넷 코리아 이정현 기자 도널드 트럼프 미국 대통령 푸틴 러시아 대통령 초상화 새기다 황금 아이폰 출시 하다 화제 모으다 러시아 업체 캐비아 이번 태양광 충전 가능하다 아이폰 를 판매 하다 매체 더버지 시간 보도 하다 업체 주로 아이폰 애플 워치 등 제품 금 장식 개조 하다 고가 판매 하다 업체 아이폰 이름 아이폰 테슬라 이 제품 면 태양 전지 패널 있다 스마트폰 뒤지다 놓다 하다 충전 되어다 보조 배터리 형태 사용 하다 수 있다 이 업체 아이폰 직접 만들다 것 아니다 때문 아이폰 탑재 되다 태양광 충전 기능 아이폰 직접 내장 되어다 있다 형태 아니다 아이폰 바깥 쪽 케이스 형태 탑재 되다 또 캐비아 그 동안 제공 하다 제품 마찬가지 고급스럽다 움 대화 하다 위해 노력 하다 아이폰 면 모서리 후 카메라 테두리 등 곳곳 금 장식 하다 제품 대다 한정 생산 되다 모델 아이폰 테슬라 가격 달러 약 만 원 모델 경우 달러 약 만


#### 불용어 처리 - 분류에 사용하지 않을 단어들을 제외합니다.
- 불용어 목록 파일 : `../data/korean_stopwords.csv`
1. 불용어 파일을 읽어와서 불용어 목록 배열을 생성합니다.

In [9]:
df = pd.read_csv('../data/korean_stopwords.csv')
df['불용어'] = df['불용어'].str.strip()
stopwords = df['불용어'].to_numpy()

print('불용어 목록 :',stopwords)

불용어 목록 : ['이' '있' '하' '것' '들' '그' '되' '수' '이' '보' '않' '없' '나' '사람' '주' '아니' '등' '같'
 '우리' '때' '년' '가' '한' '지' '대하' '오' '말' '일' '그렇' '위하' '때문' '그것' '두' '말하'
 '알' '그러나' '받' '못하' '일' '그런' '또' '문제' '더' '사회' '많' '그리고' '좋' '크' '따르' '중'
 '나오' '가지' '씨' '시키' '만들' '지금' '생각하' '그러' '속' '하나' '집' '살' '모르' '적' '월' '데'
 '자신' '안' '어떤' '내' '경우' '명' '생각' '시간' '그녀' '다시' '이런' '앞' '보이' '번' '나' '다른'
 '어떻' '여자' '개' '들' '사실' '이렇' '점' '싶' '말' '정도' '좀' '원' '잘' '통하' '소리' '놓']


2. 불용어 목록을 사용하여, 불용어를 제외한 본문만 추출합니다.

In [10]:
for i, news in enumerate(X):
    clean_words = [] 
    for word in nltk.tokenize.word_tokenize(news): # 뉴스본문을 단어단위로 사용합니다.
        if word not in stopwords: # 불용어 제외
            clean_words.append(word)
    X[i] = ' '.join(clean_words)

print(X[0])

태양광 충전 되다 아이폰 테슬라 디넷 코리아 이정현 기자 도널드 트럼프 미국 대통령 푸틴 러시아 대통령 초상화 새기다 황금 아이폰 출시 하다 화제 모으다 러시아 업체 캐비아 이번 태양광 충전 가능하다 아이폰 를 판매 하다 매체 더버지 보도 하다 업체 주로 아이폰 애플 워치 제품 금 장식 개조 하다 고가 판매 하다 업체 아이폰 이름 아이폰 테슬라 제품 면 태양 전지 패널 있다 스마트폰 뒤지다 놓다 하다 충전 되어다 보조 배터리 형태 사용 하다 있다 업체 아이폰 직접 만들다 아니다 아이폰 탑재 되다 태양광 충전 기능 아이폰 직접 내장 되어다 있다 형태 아니다 아이폰 바깥 쪽 케이스 형태 탑재 되다 캐비아 동안 제공 하다 제품 마찬가지 고급스럽다 움 대화 하다 위해 노력 하다 아이폰 면 모서리 후 카메라 테두리 곳곳 금 장식 하다 제품 대다 한정 생산 되다 모델 아이폰 테슬라 가격 달러 약 만 모델 달러 약 만


### tf-idf 벡터화
- X 데이터(뉴스 본문 데이터)를 TfidfVectorizer를 사용하여 벡터화 합니다.

In [11]:
transformer = TfidfVectorizer()
X_tfidf = transformer.fit_transform(X)


print(f'뉴스 본문의 개수 : {X_tfidf.shape[0]}, 단어의 종류 : {X_tfidf.shape[1]}')

뉴스 본문의 개수 : 1600, 단어의 종류 : 23579


In [12]:
# 모든 뉴스 본문의 단어들 확인
features = transformer.get_feature_names()
features

['가가',
 '가감',
 '가게',
 '가격',
 '가격표',
 '가결',
 '가계',
 '가계도',
 '가계부채',
 '가공',
 '가공품',
 '가교',
 '가구',
 '가금류',
 '가긍',
 '가기',
 '가까스로',
 '가까워지다',
 '가까이',
 '가깝다',
 '가꾸다',
 '가끔',
 '가나',
 '가난',
 '가난하다',
 '가내',
 '가누다',
 '가늘다',
 '가늘어지다',
 '가늠',
 '가능',
 '가능성',
 '가능하다',
 '가다',
 '가다가',
 '가다듬다',
 '가닥',
 '가담',
 '가덕도',
 '가델',
 '가도',
 '가독',
 '가동',
 '가두다',
 '가드레일',
 '가득',
 '가득하다',
 '가든',
 '가든스',
 '가디',
 '가디건',
 '가디언',
 '가뜩이나',
 '가라',
 '가라앉다',
 '가라앉히다',
 '가랑비',
 '가래',
 '가량',
 '가려움',
 '가려움증',
 '가려지다',
 '가려진',
 '가령',
 '가로',
 '가로등',
 '가로막',
 '가로막다',
 '가로막히다',
 '가로세로',
 '가로수길',
 '가로지르다',
 '가루',
 '가루약',
 '가르다',
 '가르시아',
 '가르치다',
 '가리다',
 '가리왕산',
 '가리키다',
 '가림',
 '가마',
 '가맹',
 '가면',
 '가명',
 '가무',
 '가문',
 '가물',
 '가뭄',
 '가미',
 '가바',
 '가방',
 '가버리다',
 '가벼워지다',
 '가변',
 '가볍다',
 '가보',
 '가보다',
 '가복',
 '가부장',
 '가브리엘',
 '가비',
 '가사',
 '가산',
 '가상',
 '가상세계',
 '가상화',
 '가석방',
 '가설',
 '가성',
 '가세',
 '가속',
 '가속도',
 '가수',
 '가스',
 '가스레인지',
 '가스텔럼',
 '가스통',
 '가슴',
 '가슴샘',
 '가습기',
 '가시',
 '가시거리',
 '가시꽃',
 '가시나무',
 '가압류',
 '가액',
 '가야',


In [13]:
# tf-idf 벡터화 결과 확인
tdidf_df = pd.DataFrame(X_tfidf.toarray(), index=[f'뉴스{i}' for i in range(X_tfidf.shape[0])], columns=features)
tdidf_df.head()

,가가,가감,가게,가격,가격표,가결,가계,가계도,가계부채,가공,...,힘껏,힘내다,힘드다,힘들다,힘쓰다,힘없다,힘자랑,힘주다,힘차다,힙합
뉴스0,0.0,0.0,0.000000,0.040564,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
뉴스1,0.0,0.0,0.048474,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
뉴스2,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
뉴스3,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.033117,0.0,0.0,0.0,0.0,0.0
뉴스4,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0


In [20]:
x_train, x_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.3, random_state=777, stratify=y)

## 뉴스 카테고리 분류하기
### ML-Naive Bayes
1. 모델 생성 및 학습

In [21]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB(alpha=1.0)
model.fit(x_train, y_train)

MultinomialNB()

2. 모델 평가

In [22]:
model.score(x_train, y_train), model.score(x_test, y_test) # 정확도

(0.9517857142857142, 0.8479166666666667)

3. 뉴스 분류 예측
- 예측에 사용할 뉴스 본문 데이터 또한, 앞의 정제과정을 똑같이 수행해야 합니다.
- 데이터 정제 과정을 **함수**로 생성하고, 필요할때마다 사용합니다.

In [25]:
def clean_news(documents):
    # 불용어를 미리 설정합니다
    df = pd.read_csv('../data/korean_stopwords.csv')
    df['불용어'] = df['불용어'].str.strip()
    stopwords = df['불용어'].to_numpy()
    
    for i, document in enumerate(documents):
        # 한글 데이터만 추출합니다.
        document = re.sub(r'[^ ㄱ-ㅣ가-힣]', '', document) #특수기호 제거, 정규 표현식

        # konlpy를 사용해서, 형태소 단위로 문장을 분석합니다.
        # 어간 추출을 통해 명사, 동사, 형용사만 추출합니다.
        okt = konlpy.tag.Okt()
        clean_words = []
        for word in okt.pos(document, stem=True): # 어간 추출
            if word[1] in ['Noun', 'Verb', 'Adjective']: # 명사, 동사, 형용사 추출
                clean_words.append(word[0])
        document = ' '.join(clean_words)
        
        # 설정한 불용어를 제외합니다.
        clean_words = [] 
        for word in nltk.tokenize.word_tokenize(document): 
            if word not in stopwords: #불용어 제거
                clean_words.append(word)
        documents[i] = ' '.join(clean_words)  

    return documents

- 새로운 뉴스를 작성해보고, 어떻게 분류되는지 확인합니다.

In [27]:
news_test = np.array([
    '삼성전자 주가가 90000원까지 상한가'
])
news_test = clean_news(news_test) #텍스트 정제
news_test = transformer.transform(news_test) #단어 카운트 가중치

y_pred = model.predict(news_test)
y_pred_prob = model.predict_proba(news_test)

print(f'예측 분류 : {labels[y_pred[0]]} \n확률 : {y_pred_prob.max()}')

예측 분류 : 경제 
확률 : 0.28397106682211515


reference) 토닥토닥 파이썬 - 텍스트를 위한 머신러닝

https://wikidocs.net/book/2859